# Determine global irrigation topology
## which HRUs are served by a reservoir 

Inne Vanderkelen - March 2021


In [1]:
# import modules
# add path where utils modules are located to python path
import numpy as np
import geopandas as gpd
import pandas as pd
import pickle

import warnings
warnings.filterwarnings('ignore')

# import own functions
import pfaf.pfafstetter as pfaf # decode package from Naoki, see https://github.com/nmizukami/pfaf_decode 
from utils_plotting import *
from utils_irrigtopo import *


# define data directory 
data_dir = 'C:\\Users\\ivand\\OneDrive - Vrije Universiteit Brussel/PhD/3_reservoir_release/data_for_mizuroute/'

# Settings: 
outlet_threshold = 700e3 # in m
tributary_threshold = 100e3 # m

# calculate or load reservoir dependency 
calc_res_dependency = True # if True: Calculate, if False: load

## Calculate or load reservoir dependencies

In [2]:
%%time

if calc_res_dependency: 

    # load global river segments
    river_grand_attrs = gpd.read_file(data_dir+'river_with_lake_flag4_reorder/river_with_grand.shp')

    pfaf_reservoirs = list(river_grand_attrs.loc[river_grand_attrs['H06_purpos']==1,'PFAF'].values)    
    
    river_shp = river_grand_attrs
    # get their corresponding outlets
    pfaf_outlets = get_outlets(river_shp,pfaf_reservoirs,outlet_threshold)

    # get dependent reservoirs per segment
    seg_dependency_dict = get_seg_dependency(pfaf_reservoirs, pfaf_outlets, river_shp, tributary_threshold)

    # get weights of dependend reservoirs per segment
    weights_dict = get_weights_per_seg(seg_dependency_dict, river_shp, pfaf_reservoirs,  weigh_smax_with_nseg=True)

    # get depenent segments per reservoir and corresponding weights
    res_dependency_dict = get_res_dependency_and_weights(pfaf_reservoirs, seg_dependency_dict, weights_dict)

    # save dependency dict as a pickle
    f = open(data_dir+"irrigation_topology/res_dependency_HDMA_outletthres_"+str(int(outlet_threshold*10**-3))+"km_tribthres_"+str(int(tributary_threshold*10**-3))+"km.pkl","wb")
    pickle.dump(res_dependency_dict,f)
    f.close()
    
    f = open(data_dir+"irrigation_topology/seg_dependency_HDMA_outletthres_"+str(int(outlet_threshold*10**-3))+"km_tribthres_"+str(int(tributary_threshold*10**-3))+"km.pkl","wb")
    pickle.dump(seg_dependency_dict,f)
    f.close()
    
else: 
    print('load reservoir and segment depenency')
    print('with outlet threshold of '+str(int(outlet_threshold*10**-3))+' km and tributary threshold of '+str(int(tributary_threshold*10**-3))+' km')
    # open res_dependency pickle 
    with open(data_dir+"/irrigation_topology/res_dependency_HDMA_outletthres_"+str(int(outlet_threshold*10**-3))+"km_tribthres_"+str(int(tributary_threshold*10**-3))+"km.pkl", 'rb') as handle:
        res_dependency_dict = pickle.load(handle)
        
    with open(data_dir+"/irrigation_topology/seg_dependency_HDMA_outletthres_"+str(int(outlet_threshold*10**-3))+"km_tribthres_"+str(int(tributary_threshold*10**-3))+"km.pkl", 'rb') as handle:
        seg_dependency_dict = pickle.load(handle)

-------- Searching for outlets --------

processing 1 of 484
reservoir: 2524449
reservoir reached

processing 2 of 484
reservoir: 2398995
river mouth reached

processing 3 of 484
reservoir: 2924743
length threshold exceeded

processing 4 of 484
reservoir: 243883
reservoir reached

processing 5 of 484
reservoir: 2533149
reservoir reached

processing 6 of 484
reservoir: 208945
length threshold exceeded

processing 7 of 484
reservoir: 2533385
river mouth reached

processing 8 of 484
reservoir: 2086821
length threshold exceeded

processing 9 of 484
reservoir: 208999
length threshold exceeded

processing 10 of 484
reservoir: 283367
length threshold exceeded

processing 11 of 484
reservoir: 2525365
river mouth reached

processing 12 of 484
reservoir: 25335145
river mouth reached

processing 13 of 484
reservoir: 252447
reservoir reached

processing 14 of 484
reservoir: 2516223
river mouth reached

processing 15 of 484
reservoir: 253724
river mouth reached

processing 16 of 484
reservoir: 2549

KeyboardInterrupt: 

## Apply topology on HRUs (for now only simplified ones)

In [4]:
# laod HRUS

hru_simplify = gpd.read_file("C:\\Users\\ivand\\OneDrive - Vrije Universiteit Brussel/PhD/3_reservoir_release/water_demand/mizuRoute_data/HDMA_catchment/hdma_global_catch_simplify.gpkg")
hrus_shp = hru_simplify

# calculate number of dependend reservoirs per hru. 
def count_res(row): 
    
    pfaf = row.PFAF
    if pfaf in seg_dependency_dict.keys():
        return len(seg_dependency_dict[pfaf])
    else:
          return 0

hrus_shp['n_res'] = hrus_shp.apply(lambda row: count_res(row),axis=1)

hrus_shp.to_file("C:\\Users\\ivand\\OneDrive - Vrije Universiteit Brussel/PhD/3_reservoir_release/water_demand/mizuRoute_data/HDMA_catchment/hru_simplify_nres.gpkg", driver="GPKG")


KeyboardInterrupt: 